In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import normalize
from sklearn.model_selection import train_test_split

In [ ]:
URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'

# Use pandas excel reader
df = pd.read_excel(URL)

In [ ]:
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [ ]:
train, test = train_test_split(df, test_size=0.2)

In [ ]:
def process_y(data):
  y1 = data.pop('Y1').values
  y2 = data.pop('Y2').values
  return y1, y2

In [ ]:
#Y will be a tuple of arrays (y1,y2) where y1 = [] and y2 = []
train_y = process_y(train)
train_x = train.values

test_y = process_y(test)
test_x = test.values

In [ ]:
#This is the y1 output
train_y[0].shape

(614,)

In [ ]:
train_x = normalize(train_x)
test_x = normalize(test_x)

In [ ]:
input = Input(shape=(len(train.columns),))
dense_1 = Dense(128, activation='relu')(input)
dense_2 = Dense(128, activation='relu')(dense_1)

#You name the output layers, so you can indidually assign them their loss function and metrics when compiling their model (By using their name)
output_1 = Dense(1, name='y1_output')(dense_2)

dense_3 = Dense(64, activation='relu')(dense_2)

output_2 = Dense(1, name='y2_output')(dense_3)

model = Model(inputs=input, outputs=[output_1, output_2])

In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 128)          1152        input_7[0][0]                    
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 128)          16512       dense_24[0][0]                   
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 64)           8256        dense_25[0][0]                   
____________________________________________________________________________________________

In [ ]:
opt = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=opt,
              loss={
                  'y1_output': 'mse',
                  'y2_output': 'mse'
              },
              metrics={
                  'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                  'y2_output': tf.keras.metrics.RootMeanSquaredError()
              })

In [ ]:
hist = model.fit(train_x, train_y, epochs=200, batch_size=10)

Epoch 1/200
62/62 [==============================] - 0s 4ms/step - loss: 36.7734 - y1_output_loss: 19.4051 - y2_output_loss: 17.3683 - y1_output_root_mean_squared_error: 4.4051 - y2_output_root_mean_squared_error: 4.1675
Epoch 2/200
62/62 [==============================] - 0s 4ms/step - loss: 36.9154 - y1_output_loss: 19.3360 - y2_output_loss: 17.5794 - y1_output_root_mean_squared_error: 4.3973 - y2_output_root_mean_squared_error: 4.1928
Epoch 3/200
62/62 [==============================] - 0s 4ms/step - loss: 35.7910 - y1_output_loss: 19.0598 - y2_output_loss: 16.7312 - y1_output_root_mean_squared_error: 4.3658 - y2_output_root_mean_squared_error: 4.0904
Epoch 4/200
62/62 [==============================] - 0s 4ms/step - loss: 36.5789 - y1_output_loss: 19.3528 - y2_output_loss: 17.2261 - y1_output_root_mean_squared_error: 4.3992 - y2_output_root_mean_squared_error: 4.1504
Epoch 5/200
62/62 [==============================] - 0s 4ms/step - loss: 36.3626 - y1_output_loss: 18.9185 - y2_outp

In [ ]:
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(test_x, test_y)

5/5 [==============================] - 0s 5ms/step - loss: 28.6340 - y1_output_loss: 14.5468 - y2_output_loss: 14.0872 - y1_output_root_mean_squared_error: 3.8140 - y2_output_root_mean_squared_error: 3.7533


In [ ]:
print(loss)

28.63399887084961
